# Modelo Definitivo para % PIB do Agronegócio

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# Carregar o arquivo CSV
file_path = 'C:\\DSA_USP\\DATA_SOURCE\\ANÁLISE FINAL V05.csv'
df = pd.read_csv(file_path, sep=';', decimal=',')

#----------------------------------------------------------------------------------------
# Excluir as observações de variáveis com missing values (null e NA)
df.replace(to_replace=["null", "NULL", np.nan], value="", inplace=True)
dados = df.dropna()

# Total de observações no dataframe original
total_observacoes = len(dados)

# Total de observações após a seleção de colunas
total_observacoes_missing = len(dados)

# Percentual de observações faltantes em relação ao total de observações no dataframe
aproveitamento = (total_observacoes_missing / total_observacoes) * 100

# Criar dataframe com os resultados
resultados_df = pd.DataFrame({
    "Originais": [total_observacoes],
    "Aproveitados": [total_observacoes_missing],
    "Aproveitamento (%)": [f"{aproveitamento:.1f}"]
})

print(resultados_df)
#----------------------------------------------------------------------------------------

# One-hot encoding para os atributos 'tipoacao', 'regiao' e 'unidadeorcamentaria'
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(df[['tipoacao', 'regiao', 'unidadeorcamentaria']])

# Criar nomes de colunas personalizados
encoded_columns = encoder.get_feature_names_out(['tipoacao', 'regiao', 'unidadeorcamentaria'])

# Criar DataFrame codificado com os nomes de colunas personalizados
df_encoded = pd.DataFrame(encoded_data, columns=encoded_columns, index=df.index)

# Concatenar as colunas codificadas com o DataFrame original
df = pd.concat([df, df_encoded], axis=1)

# Remover as colunas originais 'tipoacao', 'regiao' e 'unidadeorcamentaria'
df = df.drop(['tipoacao', 'regiao', 'unidadeorcamentaria'], axis=1)

df.to_excel('C:\\DSA_USP\\DATA_SOURCE\\dados_preparados.xlsx', index=False)

# Selecionar as features relevantes
features = [
    #'Autorizado_USD',
    'Pago_USD',
    'IDH',
    'PIB',
    'Log_Pop_Por_Propriedade',
    #'Norm_Pop_Por_Propriedade',
    '%_Receber_Orientacao_Tecnica',
    'tipoacao_CAPEX',
    #'tipoacao_OPEX',
    'regiao_BR',
    'regiao_Centro-Oeste',
    'regiao_Nordeste',
    'regiao_Norte',
    'regiao_Sudeste',
    'regiao_Sul',
    'unidadeorcamentaria_CONAB',
    'unidadeorcamentaria_EMBRAPA',
    'unidadeorcamentaria_INCRA',
    'unidadeorcamentaria_MAPA',
    'unidadeorcamentaria_SFB'
]

# Preparar os dados
X = df[features]
y_PIB = df['%_PIB_Agronegocio']

X.to_excel('C:\\DSA_USP\\DATA_SOURCE\\dados_preparados_X.xlsx', index=False)

# Dividir os dados em conjuntos de treino e teste para PIB_Agronegocio
X_train, X_test, y_train, y_test = train_test_split(X, y_PIB, test_size=0.2, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Seleção de Features ---

# Utilizaar um modelo de regressão linear como base para RFE (Recursive Feature Elimination)
model_rfe = LinearRegression()
rfe = RFE(estimator=model_rfe, n_features_to_select=17, step=1)
rfe.fit(X_train_scaled, y_train)

# Selecionar as features importantes
selected_features = X_train.columns[rfe.support_]
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Normalizar novamente os dados com as features selecionadas
X_train_selected_scaled = scaler.fit_transform(X_train_selected)
X_test_selected_scaled = scaler.transform(X_test_selected)

# --- Otimização de Hiperparâmetros ---

# Criar o modelo XGBoost
model = XGBRegressor(random_state=42)

# Definir a grade de hiperparâmetros a serem otimizados
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'reg_alpha': [0, 0.1, 0.5], # Regularização L1 (Lasso)
    'reg_lambda': [0, 0.1, 0.5] # Regularização L2 (Ridge)
}

# Criar o objeto GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Treinar o modelo com a grade de hiperparâmetros
grid_search.fit(X_train_selected_scaled, y_train)

# Obter os melhores hiperparâmetros
best_params = grid_search.best_params_
print("Melhores hiperparâmetros:", best_params)

# Criar o modelo com os melhores hiperparâmetros
best_model = XGBRegressor(random_state=42, **best_params)
best_model.fit(X_train_selected_scaled, y_train)

# --- Avaliação do Modelo ---

# Fazer previsões com o modelo otimizado
y_pred = best_model.predict(X_test_selected_scaled)

# Avaliar o desempenho do modelo
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Resultados para Previsão do modelo para '% PIB Agronegocio':")
print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("R2 Score:", r2)

# --- Importância das Features ---

feature_importances = pd.DataFrame({
    'Feature': X_train_selected.columns,
    'Importância': best_model.feature_importances_
})
feature_importances_sorted = feature_importances.sort_values(by='Importância', ascending=False)
print(feature_importances_sorted)


   Originais  Aproveitados Aproveitamento (%)
0        465           465              100.0
Melhores hiperparâmetros: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'reg_alpha': 0, 'reg_lambda': 0.1}
Resultados para Previsão do modelo para '% PIB Agronegocio':
MSE: 9.142347787437232e-05
RMSE: 0.009561562522640968
MAE: 0.0035760367636491486
R2 Score: 0.8764343967963025
                         Feature  Importância
4   %_Receber_Orientacao_Tecnica     0.281825
5                 tipoacao_CAPEX     0.215948
1                            IDH     0.117489
3        Log_Pop_Por_Propriedade     0.105853
2                            PIB     0.089093
13   unidadeorcamentaria_EMBRAPA     0.051075
10                regiao_Sudeste     0.039999
8                regiao_Nordeste     0.038909
0                       Pago_USD     0.021143
15      unidadeorcamentaria_MAPA     0.010413
9                   regiao_Norte     0.009173
11                    regiao_Sul     0.008256
14     unidadeorca

Como um Pós-Doutorado em Econometria Financeira, reconheço que minha sugestão anterior não resultou em melhorias significativas no modelo. Agradeço por compartilhar esses resultados adicionais, pois eles fornecem insights importantes. Vamos analisar:

1. Consistência dos resultados:
O fato de que os resultados são idênticos para 'Populacao_Por_Propriedade', 'Log_Pop_Por_Propriedade' e 'Norm_Pop_Por_Propriedade' é notável e informativo.

2. Interpretação:
Esta consistência sugere que a relação entre a população por propriedade e o %_PIB_Agronegocio é robusta e linear dentro do intervalo dos dados observados.

3. Implicações para o modelo:
- O XGBoost está capturando eficientemente a informação contida nesta variável, independentemente da transformação aplicada.
- A natureza não-paramétrica do XGBoost permite que ele se adapte à distribuição dos dados sem necessidade de transformações explícitas.

4. Lições aprendidas:
- Transformações de variáveis nem sempre resultam em melhorias de modelos baseados em árvores, como o XGBoost.
- A importância consistente da variável (0.105853) indica que ela é um preditor estável do %_PIB_Agronegocio.

Recomendações revisadas:

1. Manter a variável original:
Dado que as transformações não trouxeram benefícios, é preferível manter 'Populacao_Por_Propriedade' em sua forma original por simplicidade e interpretabilidade.

2. Análise de threshold:
Investigue se existem pontos de corte específicos em 'Populacao_Por_Propriedade' que influenciam significativamente o %_PIB_Agronegocio.

3. Interações:
Explore interações entre 'Populacao_Por_Propriedade' e outras variáveis importantes, como IDH ou tipoacao_CAPEX. Isso pode revelar relações mais complexas.

4. Análise regional:
Examine como a importância de 'Populacao_Por_Propriedade' varia entre diferentes regiões, o que pode fornecer insights sobre diferenças estruturais no setor agrícola.

5. Modelos alternativos:
Considere testar modelos paramétricos (como regressão linear múltipla) para comparar com o XGBoost e verificar se há relações lineares que podem ser capturadas de forma mais simples.

6. Análise temporal:
Se possível, analise como a relação entre 'Populacao_Por_Propriedade' e %_PIB_Agronegocio evoluiu ao longo do tempo.

Conclusão:
Embora as transformações sugeridas não tenham melhorado o modelo, este exercício foi valioso. Ele demonstrou a robustez da relação entre população por propriedade e o PIB do agronegócio, além de confirmar a eficácia do XGBoost em capturar esta relação sem necessidade de transformações. Para futuras iterações do modelo, foque em explorar interações mais complexas e análises contextuais (regionais ou temporais) para extrair insights adicionais dos dados.

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# Carregar o arquivo CSV
file_path = 'C:\\DSA_USP\\DATA_SOURCE\\ANÁLISE FINAL V05.csv'
df = pd.read_csv(file_path, sep=';', decimal=',')

#----------------------------------------------------------------------------------------
# Excluir as observações de variáveis com missing values (null e NA)
df.replace(to_replace=["null", "NULL", np.nan], value="", inplace=True)
dados = df.dropna()

# Total de observações no dataframe original
total_observacoes = len(dados)

# Total de observações após a seleção de colunas
total_observacoes_missing = len(dados)

# Percentual de observações faltantes em relação ao total de observações no dataframe
aproveitamento = (total_observacoes_missing / total_observacoes) * 100

# Criar dataframe com os resultados
resultados_df = pd.DataFrame({
    "Originais": [total_observacoes],
    "Aproveitados": [total_observacoes_missing],
    "Aproveitamento (%)": [f"{aproveitamento:.1f}"]
})

print(resultados_df)
#----------------------------------------------------------------------------------------

# One-hot encoding para os atributos 'tipoacao' e 'regiao'
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(df[['tipoacao', 'regiao', 'unidadeorcamentaria']])

# Criar nomes de colunas personalizados
encoded_columns = encoder.get_feature_names_out(['tipoacao', 'regiao', 'unidadeorcamentaria'])

# Criar DataFrame codificado com os nomes de colunas personalizados
df_encoded = pd.DataFrame(encoded_data, columns=encoded_columns, index=df.index)

# Concatenar as colunas codificadas com o DataFrame original
df = pd.concat([df, df_encoded], axis=1)

# Remover as colunas originais 'tipoacao' e 'regiao'
df = df.drop(['tipoacao', 'regiao', 'unidadeorcamentaria'], axis=1)

df.to_excel('C:\\DSA_USP\\DATA_SOURCE\\dados_preparados.xlsx', index=False)

# Selecionar as features relevantes
features = [
    #'Autorizado_USD',
    'Pago_USD',
    'IDH',
    'PIB',
    #'Log_Pop_Por_Propriedade',
    'Norm_Pop_Por_Propriedade',
    '%_Receber_Orientacao_Tecnica',
    'tipoacao_CAPEX',
    #'tipoacao_OPEX',
    'regiao_BR',
    'regiao_Centro-Oeste',
    'regiao_Nordeste',
    'regiao_Norte',
    'regiao_Sudeste',
    'regiao_Sul',
    'unidadeorcamentaria_CONAB',
    'unidadeorcamentaria_EMBRAPA',
    'unidadeorcamentaria_INCRA',
    'unidadeorcamentaria_MAPA'
    #'unidadeorcamentaria_SFB'
]

# Preparar os dados
X = df[features]
y_PIB = df['%_PIB_Agronegocio']

X.to_excel('C:\\DSA_USP\\DATA_SOURCE\\dados_preparados_X.xlsx', index=False)

# Dividir os dados em conjuntos de treino e teste para PIB_Agronegocio
X_train, X_test, y_train, y_test = train_test_split(X, y_PIB, test_size=0.2, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Seleção de Features ---

# 1. RFE (Recursive Feature Elimination)
# Utilizando um modelo de regressão linear como base para RFE
model_rfe = LinearRegression()
rfe = RFE(estimator=model_rfe, n_features_to_select=16, step=1)
rfe.fit(X_train_scaled, y_train)

# Selecionando as features importantes
selected_features = X_train.columns[rfe.support_]
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Normalizando novamente os dados com as features selecionadas
X_train_selected_scaled = scaler.fit_transform(X_train_selected)
X_test_selected_scaled = scaler.transform(X_test_selected)

# --- Otimização de Hiperparâmetros ---

# Criar o modelo XGBoost
model = XGBRegressor(random_state=42)

# Definir a grade de hiperparâmetros a serem otimizados
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'reg_alpha': [0, 0.1, 0.5], # Regularização L1 (Lasso)
    'reg_lambda': [0, 0.1, 0.5] # Regularização L2 (Ridge)
}

# Criar o objeto GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Treinar o modelo com a grade de hiperparâmetros
grid_search.fit(X_train_selected_scaled, y_train)

# Obter os melhores hiperparâmetros
best_params = grid_search.best_params_
print("Melhores hiperparâmetros:", best_params)

# Criar o modelo com os melhores hiperparâmetros
best_model = XGBRegressor(random_state=42, **best_params)
best_model.fit(X_train_selected_scaled, y_train)

# --- Avaliação do Modelo ---

# Fazer previsões com o modelo otimizado
y_pred = best_model.predict(X_test_selected_scaled)

# Avaliar o desempenho do modelo
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Resultados para Previsão do % PIB Agronegocio:")
print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("R2 Score:", r2)

# --- Importância das Features ---
feature_importances = pd.DataFrame({
    'Feature': X_train_selected.columns,
    'Importância': best_model.feature_importances_
})
feature_importances_sorted = feature_importances.sort_values(by='Importância', ascending=False)
print(feature_importances_sorted)


   Originais  Aproveitados Aproveitamento (%)
0        465           465              100.0
Melhores hiperparâmetros: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'reg_alpha': 0, 'reg_lambda': 0.1}
Resultados para Previsão do % PIB Agronegocio:
MSE: 9.142347787437232e-05
RMSE: 0.009561562522640968
MAE: 0.0035760367636491486
R2 Score: 0.8764343967963025
                         Feature  Importância
4   %_Receber_Orientacao_Tecnica     0.281825
5                 tipoacao_CAPEX     0.215948
1                            IDH     0.117489
3       Norm_Pop_Por_Propriedade     0.105853
2                            PIB     0.089093
13   unidadeorcamentaria_EMBRAPA     0.051075
10                regiao_Sudeste     0.039999
8                regiao_Nordeste     0.038909
0                       Pago_USD     0.021143
15      unidadeorcamentaria_MAPA     0.010413
9                   regiao_Norte     0.009173
11                    regiao_Sul     0.008256
14     unidadeorcamentaria_INCRA

### Interpretação Detalhada em Linguagem Científica

#### Desempenho do Modelo

1. **Melhores Hiperparâmetros do Modelo**:
   - `learning_rate`: 0.1
   - `max_depth`: 5
   - `n_estimators`: 300
   - `reg_alpha`: 0
   - `reg_lambda`: 0.5

   Estes hiperparâmetros foram obtidos através de uma busca em grade (GridSearchCV), refletindo um modelo XGBoost bem ajustado para capturar a complexidade dos dados e evitar o sobreajuste. Um `learning_rate` de 0.1 e `max_depth` de 5 indicam um modelo com capacidade moderada de aprendizado, balanceando a complexidade do modelo e sua capacidade de generalização.

2. **Métricas de Avaliação**:
   - **MSE (Erro Quadrático Médio)**: 0.00014968975908552414
   - **RMSE (Raiz do Erro Quadrático Médio)**: 0.01223477662589408
   - **MAE (Erro Médio Absoluto)**: 0.006528412222434093
   - **R² (Coeficiente de Determinação)**: 0.797683201242513

   O MSE baixo indica que o modelo possui um erro médio quadrático pequeno entre as previsões e os valores reais. O RMSE, que é a raiz quadrada do MSE, também é baixo, indicando boa precisão do modelo. O MAE mostra um erro absoluto médio relativamente baixo. Um R² de 0.7977 indica que aproximadamente 79.77% da variabilidade na variável dependente (% do PIB do Agronegócio) é explicada pelas variáveis independentes no modelo, sugerindo um bom ajuste.

#### Importância das Variáveis

As importâncias das variáveis, calculadas pelo modelo XGBoost, destacam as variáveis que mais contribuíram para a previsão do % do PIB do Agronegócio. Aqui está a lista das variáveis em ordem de importância:

1. **regiao_BR**: 0.484863
2. **tipoacao_CAPEX**: 0.107682
3. **regiao_Centro-Oeste**: 0.102609
4. **IDH**: 0.064739
5. **regiao_Nordeste**: 0.054166
6. **PIB**: 0.052690
7. **regiao_Norte**: 0.044522
8. **regiao_Sudeste**: 0.034162
9. **regiao_Sul**: 0.022711
10. **Pago**: 0.017980
11. **Autorizado**: 0.013875
12. **tipoacao_OPEX**: 0.000000

A variável `regiao_BR` possui a maior importância, sugerindo que a localização regional no Brasil é um fator significativo na contribuição do agronegócio ao PIB. Em seguida, variáveis como `tipoacao_CAPEX` e `regiao_Centro-Oeste` também têm importâncias consideráveis, indicando a relevância dos investimentos em capital e das particularidades regionais.

### Resumo em Linguagem de Negócio

#### Desempenho do Modelo

- **Modelo Otimizado**: Utilizamos um modelo XGBoost ajustado com hiperparâmetros ótimos para prever a contribuição do agronegócio ao PIB.
- **Alta Precisão**: Nosso modelo apresenta um MSE de 0.0001497, RMSE de 0.0122, MAE de 0.0065 e um R² de 0.798, indicando que o modelo explica aproximadamente 79.8% da variabilidade na contribuição do agronegócio ao PIB.

#### Importância das Variáveis

- **Influência Regional**: A variável `regiao_BR` é a mais influente, destacando a importância da localização regional no Brasil para o agronegócio.
- **Investimentos em Capital (CAPEX)**: Os investimentos em infraestrutura e capital são cruciais para o crescimento do PIB no agronegócio.
- **Contribuições Regionais Específicas**: Regiões como Centro-Oeste e Nordeste têm uma contribuição significativa, mostrando a importância dessas áreas na economia agrícola.

#### Insights para Ação

- **Foco Regional**: Estratégias e políticas devem considerar as diferenças regionais para maximizar a eficácia das intervenções no agronegócio.
- **Investimento em Infraestrutura**: Aumentar os investimentos de capital (CAPEX) pode significativamente aumentar a contribuição do agronegócio ao PIB.
- **Atenção às Regiões-Chave**: Regiões como Centro-Oeste e Nordeste devem receber atenção especial para fomentar o crescimento agrícola.

Esses insights podem guiar a formulação de políticas e estratégias empresariais mais informadas e direcionadas para fortalecer o setor de agronegócio e, consequentemente, aumentar sua contribuição para a economia do Brasil.